# Faults sources within an area source
Given an area source and a set of faults we find the faults whose trace intersects the polygon of the area source and computes the percentage of the fault trace inside the polygon.

%%html
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<p style="font-size:60%;">
<a href="javascript:code_toggle()">[Toggle Code]</a>
<a target="_blank" href="./../project/project_set_params_gui.ipynb#">[Set params]</a
</p>

In [ ]:
%matplotlib inline
import os
import re
import sys
import numpy
from prettytable import PrettyTable
from oqmbt.oqt_project import OQtProject
from oqmbt.tools.geo import get_idx_points_inside_polygon, get_line_inside_polygon

## Info

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
print('Reading project file: %s' % (project_pickle_filename))
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
try:
    area_source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    area_source_ids_list = ['10']
print('Processing area source with ID:', area_source_ids_list[0])

## Selected faults

In [ ]:
from prettytable import PrettyTable

idx_faults_in_polygon = []
frac_faults_in_polygon = {}
#
# Setting table header
p = PrettyTable(["ID","Name","Fraction","Slip rate", "RI"])
p.align["ID"] = 'l'
p.align["Name"] = 'l'
p.align["Fraction"] = 'r'
p.align["Slip rate"] = 'r'
p.align["RI"] = 'r'
p.padding_width = 1
p.float_format = '.2'
#
# 
assert len(area_source_ids_list) == 1
area_source = model.sources[area_source_ids_list[0]]
area_source.ids_faults_inside = {}
#
# Processing faults
for key in model.sources:
    fault = model.sources[key]
    if fault.source_type == 'SimpleFaultSource':
        plons = []
        plats = []
        pdeps = []
        #
        # Create lists with the coordinates of the points defining the trace
        for pnt in fault.trace.points:
            plons.append(pnt.longitude)
            plats.append(pnt.latitude)
            pdeps.append(pnt.depth)

        # Find points on the trace within the polygon
        iii = get_idx_points_inside_polygon(plons, 
                                            plats, 
                                            area_source.polygon.lons, 
                                            area_source.polygon.lats, 
                                            numpy.arange(0, len(plons)+1))
        #
        #
        frac = get_line_inside_polygon(plons, 
                                       plats, 
                                       area_source.polygon.lons, 
                                       area_source.polygon.lats)
        #
        #
        if frac is not None and frac > 0.0 and hasattr(fault, 'slip_rate'):
            idx_faults_in_polygon.append(fault.source_id)
            frac_faults_in_polygon[fault.source_id] = frac
            recurrence = fault.recurrence if hasattr(fault, 'recurrence') else None
            slip_rate = fault.slip_rate if hasattr(fault, 'slip_rate') else None
            p.add_row([fault.source_id, fault.name, frac, slip_rate, recurrence])
            #
            # updating information for the current area source
            if hasattr(area_source, 'ids_faults_inside'):
                area_source.ids_faults_inside[fault.source_id] = frac
            else:
                area_source.ids_faults_inside = {fault.source_id: frac} 
        #
        #
        if frac is not None and frac > 0.5:
            fault.id_intersecting_area = area_source_ids_list[0]
            
print('Faults intersecting the area source %s\n' % (area_source_ids_list[0]))     
print(p)

model.sources[area_source_ids_list[0]] = area_source
oqtkp.models[model_id] = model
oqtkp.save()